In [1]:
import json

from tqdm import tqdm
import tweepy
from dotenv import load_dotenv
import os

from tweepy import TweepyException

load_dotenv()  # take environment variables from .env.

True

In [2]:
DATA_PATH = 'data'
YEARS = [2016, 2020]

In [3]:
for year in YEARS:
    with open(os.path.join(DATA_PATH, f'polls_{year}_labeled.json')) as f:
        polls = json.load(f)
    author_ids = list(map(lambda x: x['meta']['author_id'], polls))
    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json'), 'w+') as f:
        json.dump(author_ids, f)

In [4]:
auth = tweepy.OAuth2BearerHandler(
    bearer_token= os.environ['BEARER'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
exception_users = dict()

In [8]:

for year in YEARS:
    out_path = os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl')
    existing_users = set()
    if os.path.exists(out_path):
        with open(out_path) as f:
            existing_users.update(k for l in f for k in json.loads(l))
            print(f'found {len(existing_users)} scraped users')

    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json')) as f:
        author_ids = [id for id in json.load(f) if id not in existing_users]

    with open(os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl'), 'a+', encoding='utf8') as f:
        for author_id in tqdm(author_ids, desc='checking followers of poll authors'):
            follower_ids = []
            try:
                for page in tweepy.Cursor(api.get_follower_ids, user_id = author_id, count = 5000).pages():
                    follower_ids.extend(page)
                f.write(json.dumps({author_id:follower_ids}) + '\n')
            except TweepyException as e:
                print(e)
                print(author_id)
                exception_users[id]=e


found 59 scraped users


checking followers of poll authors:   2%|▏         | 1/50 [11:43<9:34:29, 703.46s/it]

401 Unauthorized
Not authorized.
349240397


checking followers of poll authors:   4%|▍         | 2/50 [11:43<3:51:51, 289.81s/it]

401 Unauthorized
Not authorized.
155939890


...

checking followers of poll authors:  61%|██████    | 351/580 [7:00:33<13:25:52, 211.15s/it]Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 893
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 894
Rate limit reached. Sleeping for: 8

404 Not Found
34 - Sorry, that page does not exist.
1273811445269106688


checking followers of poll authors:  83%|████████▎ | 484/580 [22:31:45<25:30, 15.94s/it]   

404 Not Found
34 - Sorry, that page does not exist.
1216703337317457921


checking followers of poll authors: 100%|██████████| 580/580 [24:16:55<00:00, 150.72s/it]


In [ ]:
# client = tweepy.Client(bearer_token=os.environ['BEARER'],
#                        wait_on_rate_limit=True)
# author_id = author_ids[0]
# user_fields = ['id']
# response = client.get_users_followers(id=author_id, user_fields=user_fields, max_results=1000)
# one_user = response.data[0]
# one_user.data
# next_token = response.meta['next_token']